In [2]:
!module load google-cloud-sdk

In [4]:
!gcloud auth application-default set-quota-project ee-liuliuzheng1208

/bin/bash: gcloud: command not found


In [1]:
import json
import itertools
import ee
import os
from datetime import datetime, timedelta, time
import pandas as pd
import numpy as np
import time as sleeptime #otherwise interferes with datetime.time
from functools import reduce
import math

In [2]:
ee.Authenticate()

True

In [3]:
ee.Initialize(project='ee-liuliuzheng1208')

os.chdir('/gpfs/data1/oshangp/liuz/sesync')

In [4]:
os.getcwd()

'/gpfs/data1/oshangp/liuz/sesync'

In [5]:
input_folder = './data/processed/pre_weather'
output_folder = './data/processed/nldas_weather'

In [6]:
def getFiles():
    #Get list of all files that have not yet been processed
    fs = os.listdir(input_folder)
    fs = [f for f in fs if f[-3:] == 'csv']
    
    done = os.listdir(output_folder)
    
    fs = [f for f in fs if f not in done]
    
    fs.sort(reverse=True)
   
    return(fs)

In [7]:
len(getFiles())

2114

In [8]:
#Need
# specific humidity
# temperature
# pressure
# shorwave radiation
# longwave radiation
# wind u component
# wind v component

def chunker(iter, size):
    chunks = [];
    if size < 1:
        raise ValueError('Chunk size must be greater than 0.')
    for i in range(0, len(iter), size):
        chunks.append(iter[i:(i+size)])
    return chunks

In [72]:
def getDataValues(feats, img, var):
    #First, break feats into lists of lists, each with len < 4500
    feats = chunker(feats, 4500)
    
    allres = []
    for feat in feats:
        res = img.reduceRegions(ee.FeatureCollection(feat), reducer=ee.Reducer.first()).getInfo()
        allres = allres + res['features']
    
    df = pd.DataFrame([x['properties'] for x in allres])
    return(df['first'].values)

In [9]:
def getData(feats, img, var):
    #First, break feats into lists of lists, each with len < 4500
    feats = chunker(feats, 4500)
    
    allres = []
    for feat in feats:
        res = img.reduceRegions(ee.FeatureCollection(feat), reducer=ee.Reducer.first()).getInfo()
        allres = allres + res['features']
    
    df = pd.DataFrame([x['properties'] for x in allres])
    df = df.rename(columns={'first': var})
    return(df)

In [10]:
dat = pd.read_csv(os.path.join('./data/processed/pre_weather','id_utc_20170428.csv'),dtype = {'yyyymmdd':str,'hour':str,'id':str})

In [11]:
todos = getFiles()

In [12]:
todos

['id_utc_20160921.csv',
 'id_utc_20160920.csv',
 'id_utc_20160919.csv',
 'id_utc_20160918.csv',
 'id_utc_20160917.csv',
 'id_utc_20160916.csv',
 'id_utc_20160915.csv',
 'id_utc_20160914.csv',
 'id_utc_20160913.csv',
 'id_utc_20160912.csv',
 'id_utc_20160911.csv',
 'id_utc_20160910.csv',
 'id_utc_20160909.csv',
 'id_utc_20160908.csv',
 'id_utc_20160907.csv',
 'id_utc_20160906.csv',
 'id_utc_20160905.csv',
 'id_utc_20160904.csv',
 'id_utc_20160903.csv',
 'id_utc_20160902.csv',
 'id_utc_20160901.csv',
 'id_utc_20160831.csv',
 'id_utc_20160830.csv',
 'id_utc_20160829.csv',
 'id_utc_20160828.csv',
 'id_utc_20160827.csv',
 'id_utc_20160826.csv',
 'id_utc_20160825.csv',
 'id_utc_20160824.csv',
 'id_utc_20160823.csv',
 'id_utc_20160822.csv',
 'id_utc_20160821.csv',
 'id_utc_20160820.csv',
 'id_utc_20160819.csv',
 'id_utc_20160818.csv',
 'id_utc_20160817.csv',
 'id_utc_20160816.csv',
 'id_utc_20160815.csv',
 'id_utc_20160814.csv',
 'id_utc_20160813.csv',
 'id_utc_20160812.csv',
 'id_utc_2016081

In [13]:
f = todos[0] #id_utc_20160921.csv

In [59]:
dat = pd.read_csv(os.path.join(input_folder,f),dtype = {'yyyymmdd':str,'hour':str,'id':str})

In [60]:
dat

,id,tweet_created_at_int,lat,lon,yyyymmdd,hour
0,778684579775336400,1474487845000,36.747800,-119.772000,20160921,19
1,778684579775336400,1474487845000,36.747800,-119.772000,20160921,19
2,778729035245301800,1474498444000,36.747800,-119.772000,20160921,22
3,778582867760844800,1474463595000,42.325422,-83.051687,20160921,13
4,778688899711328300,1474488875000,42.330570,-83.048553,20160921,20
...,...,...,...,...,...,...
17169,778741475441909800,1474501410000,40.669308,-73.715026,20160921,23
17170,778742360532332500,1474501621000,43.222190,-88.004265,20160921,23
17171,778743253650649100,1474501834000,42.565870,-71.746990,20160921,23
17172,778743467484655600,1474501885000,41.650270,-83.567550,20160921,23


In [61]:
daydat = pd.DataFrame()

In [62]:
hour = dat.hour.unique().tolist()[0]

In [63]:
sel = dat[dat.hour == hour] 
img = ee.Image('NASA/NLDAS/FORA0125_H002/A' + f[7:11] + f[11:13] + f[13:15] + '_' + hour.zfill(2) + '00')
temp = img.select('temperature')
speh = img.select('specific_humidity')
pres = img.select('pressure')
prcp = img.select('total_precipitation')
srad = img.select('shortwave_radiation')
lrad = img.select('longwave_radiation')
wndu = img.select('wind_u')
wndv = img.select('wind_v')

In [64]:
sel

,id,tweet_created_at_int,lat,lon,yyyymmdd,hour
0,778684579775336400,1474487845000,36.747800,-119.772000,20160921,19
1,778684579775336400,1474487845000,36.747800,-119.772000,20160921,19
5,778678845989802000,1474486478000,42.349257,-71.067844,20160921,19
6,778678845989802000,1474486478000,42.349257,-71.067844,20160921,19
49,778671426123419600,1474484709000,29.760427,-95.369803,20160921,19
...,...,...,...,...,...,...
17119,778683468158951400,1474487580000,25.728268,-80.433307,20160921,19
17120,778683543585030100,1474487598000,35.097401,-106.660915,20160921,19
17121,778683543585030100,1474487598000,35.097401,-106.660915,20160921,19
17122,778684223066550300,1474487760000,28.091667,-82.382500,20160921,19


In [65]:
feats = []
for i,v in sel.iterrows():
    lon = v['lon']
    lat = v['lat']
    pt = ee.Geometry.Point(lon, lat)
    feat = ee.Feature(pt, {'tweet_created_at_int': v['tweet_created_at_int'], 'id': str(v['id'])})
    feats.append(feat)

In [73]:
for var, img in [('temp',temp),
                 ('speh',speh),
                 ('pres',pres),
                 ('prcp',prcp),
                 ('srad',srad),
                 ('lrad',lrad),
                 ('wndu',wndu),
                 ('wndv',wndv)
                ]:
    sel.loc[:,var] = getDataValues(feats, img, var)

res = sel[['id','tweet_created_at_int','temp',
       'speh', 'pres', 'prcp', 'srad', 'lrad', 'wndu', 'wndv']]

/gpfs/data1/oshangp/liuz/gs7_gpd/lib/python3.7/site-packages/pandas/core/indexing.py:1773: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(ilocs[0], value, pi)


In [77]:
res

,id,tweet_created_at_int,temp,speh,pres,prcp,srad,lrad,wndu,wndv
0,778684579775336400,1474487845000,31.97,0.004299,100284.10,0.0000,683.537,397.53,2.03,-0.90
1,778684579775336400,1474487845000,31.97,0.004299,100284.10,0.0000,683.537,397.53,2.03,-0.90
5,778678845989802000,1474486478000,28.33,0.007667,101914.18,0.0000,571.345,354.72,1.89,-3.07
6,778678845989802000,1474486478000,28.33,0.007667,101914.18,0.0000,571.345,354.72,1.89,-3.07
49,778671426123419600,1474484709000,32.77,0.018288,101315.14,0.0000,886.065,437.89,-3.13,1.27
...,...,...,...,...,...,...,...,...,...,...
17119,778683468158951400,1474487580000,30.48,0.017591,101521.86,0.3292,727.057,439.22,-0.37,0.81
17120,778683543585030100,1474487598000,30.08,0.006991,84959.94,0.0000,787.921,383.43,0.22,1.11
17121,778683543585030100,1474487598000,30.08,0.006991,84959.94,0.0000,787.921,383.43,0.22,1.11
17122,778684223066550300,1474487760000,32.07,0.017130,101319.62,0.0476,586.641,456.94,0.50,-0.94


In [25]:
temp_res = getData(feats, temp, 'temp')
speh_res = getData(feats, speh, 'speh')
pres_res = getData(feats, pres, 'pres')
prcp_res = getData(feats, prcp, 'prcp')
srad_res = getData(feats, srad, 'srad')
lrad_res = getData(feats, lrad, 'lrad')
wndu_res = getData(feats, wndu, 'wndu')
wndv_res = getData(feats, wndv, 'wndv')

In [42]:
sel

,id,tweet_created_at_int,lat,lon,yyyymmdd,hour
0,778684579775336400,1474487845000,36.747800,-119.772000,20160921,19
1,778684579775336400,1474487845000,36.747800,-119.772000,20160921,19
5,778678845989802000,1474486478000,42.349257,-71.067844,20160921,19
6,778678845989802000,1474486478000,42.349257,-71.067844,20160921,19
49,778671426123419600,1474484709000,29.760427,-95.369803,20160921,19
...,...,...,...,...,...,...
17119,778683468158951400,1474487580000,25.728268,-80.433307,20160921,19
17120,778683543585030100,1474487598000,35.097401,-106.660915,20160921,19
17121,778683543585030100,1474487598000,35.097401,-106.660915,20160921,19
17122,778684223066550300,1474487760000,28.091667,-82.382500,20160921,19


In [68]:
sel['temp'] = getDataSeries(feats, temp, 'temp')

0       31.97
1       31.97
2       28.33
3       28.33
4       32.77
        ...  
2829    30.48
2830    30.08
2831    30.08
2832    32.07
2833    32.07
Name: first, Length: 2834, dtype: float64

,id,tweet_created_at_int
0,778684579775336400,1474487845000
1,778684579775336400,1474487845000
2,778678845989802000,1474486478000
3,778678845989802000,1474486478000
4,778671426123419600,1474484709000
...,...,...
2829,778683468158951400,1474487580000
2830,778683543585030100,1474487598000
2831,778683543585030100,1474487598000
2832,778684223066550300,1474487760000


In [53]:
temp_res[['id','tweet_created_at_int']]

,id,tweet_created_at_int
0,778684579775336400,1474487845000
1,778684579775336400,1474487845000
2,778678845989802000,1474486478000
3,778678845989802000,1474486478000
4,778671426123419600,1474484709000
...,...,...
2829,778683468158951400,1474487580000
2830,778683543585030100,1474487598000
2831,778683543585030100,1474487598000
2832,778684223066550300,1474487760000


In [52]:
sel[['id','tweet_created_at_int']].reset_index(drop=True)

,id,tweet_created_at_int
0,778684579775336400,1474487845000
1,778684579775336400,1474487845000
2,778678845989802000,1474486478000
3,778678845989802000,1474486478000
4,778671426123419600,1474484709000
...,...,...
2829,778683468158951400,1474487580000
2830,778683543585030100,1474487598000
2831,778683543585030100,1474487598000
2832,778684223066550300,1474487760000


In [57]:
temp_res[['id','tweet_created_at_int']].dtypes

id                      object
tweet_created_at_int     int64
dtype: object

In [58]:
sel[['id','tweet_created_at_int']].reset_index(drop=True).dtypes

id                      int64
tweet_created_at_int    int64
dtype: object

In [55]:
temp_res[['id','tweet_created_at_int']].compare(sel[['id','tweet_created_at_int']].reset_index(drop=True))

id                    
                    self               other
0     778684579775336400  778684579775336448
1     778684579775336400  778684579775336448
2     778678845989802000  778678845989801984
3     778678845989802000  778678845989801984
4     778671426123419600  778671426123419648
...                  ...                 ...
2829  778683468158951400  778683468158951424
2830  778683543585030100  778683543585030144
2831  778683543585030100  778683543585030144
2832  778684223066550300  778684223066550272
2833  778684223066550300  778684223066550272

[2834 rows x 2 columns]

In [36]:
res = reduce(lambda x, y: pd.merge(x, y, on=['tweet_created_at_int', 'id']), 
                [temp_res, speh_res, pres_res, prcp_res, srad_res, lrad_res, wndu_res, wndv_res])

KeyboardInterrupt: 

In [27]:
for f in todos:  
    print(datetime.now(), f)
    
    dat = pd.read_csv(os.path.join(input_folder,f),dtype = {'yyyymmdd':str,'hour':str})
    
    print(dat.shape[0])
    if dat.shape[0] == 0:
        with open(os.path.join(output_folder,f), 'w') as file:
            pass
        print("Skipping " + f)
        continue
    
    daydat = pd.DataFrame()
    for hour in dat.hour.unique().tolist():
        sel = dat[dat.hour == hour] 
        img = ee.Image('NASA/NLDAS/FORA0125_H002/A' + f[7:11] + f[11:13] + f[13:15] + '_' + hour.zfill(2) + '00')
        temp = img.select('temperature')
        speh = img.select('specific_humidity')
        pres = img.select('pressure')
        prcp = img.select('total_precipitation')
        srad = img.select('shortwave_radiation')
        lrad = img.select('longwave_radiation')
        wndu = img.select('wind_u')
        wndv = img.select('wind_v')
         
        feats = []
        for i,v in sel.iterrows():
            lon = v['lon']
            lat = v['lat']
            pt = ee.Geometry.Point(lon, lat)
            feat = ee.Feature(pt, {'tweet_created_at_int': v['tweet_created_at_int'], 'id': str(v['id'])})
            feats.append(feat)
         
        temp_res = getData(feats, temp, 'temp')
        speh_res = getData(feats, speh, 'speh')
        pres_res = getData(feats, pres, 'pres')
        prcp_res = getData(feats, prcp, 'prcp')
        srad_res = getData(feats, srad, 'srad')
        lrad_res = getData(feats, lrad, 'lrad')
        wndu_res = getData(feats, wndu, 'wndu')
        wndv_res = getData(feats, wndv, 'wndv')
        
        res = reduce(lambda x, y: pd.merge(x, y, on=['tweet_created_at_int', 'id']), 
                [temp_res, speh_res, pres_res, prcp_res, srad_res, lrad_res, wndu_res, wndv_res])
        daydat = pd.concat([daydat, res])
        print(daydat.shape)
    daydat.to_csv(os.path.join(output_folder,f), index=False)
    sleeptime.sleep(10)

2024-03-18 01:11:25.682133 id_utc_20160921.csv
17174


KeyboardInterrupt: 